In [1]:
import numpy as np

In [2]:
class Item():
    
    def __init__(self, value, weight):
        self.v = value
        self.w = weight
        self.density = value/weight
    
    def get_density(self):
        return self.density
    
    def __str__(self):
        return "value: "+str(self.v)+" weight: "+str(self.w)+" density: "+str(self.get_density())

In [69]:
n = 60
np.random.seed(2376)
I = [Item(np.random.randint(low=1, high=100), np.random.randint(low=2, high=20)) for _ in range(n)]
V = []
maxWeight = 30

In [66]:
I[0].get_density()

17.5

### 1. Полный перебо

In [70]:
def maxVal(items, maxWeight):
    if items == [] or maxWeight == 0:
        # end of tree
        result = (0, ())
    elif items[0].w > maxWeight:
        # so large
        result = maxVal(items[1:], maxWeight)
    else:
        # take or don't
        nextItem = items[0]
        withVal, withToTake = maxVal(items[1:], maxWeight - nextItem.w)
        withVal += nextItem.v
        withoutVal, withoutToTake = maxVal(items[1:], maxWeight)
        if withVal > withoutVal:
            result = (withVal, withToTake + (nextItem,))
        else:
            result = (withoutVal, withoutToTake)
    return result

In [76]:
%%timeit 
maxVal(I, maxWeight)

1.7 s ± 28.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [71]:
%%time
V = maxVal(I, maxWeight)

CPU times: total: 1.83 s
Wall time: 1.84 s


In [72]:
print(V[0])
for it in V[1]:
    print(it)

617
value: 62 weight: 3 density: 20.666666666666668
value: 94 weight: 4 density: 23.5
value: 73 weight: 5 density: 14.6
value: 90 weight: 5 density: 18.0
value: 84 weight: 3 density: 28.0
value: 69 weight: 2 density: 34.5
value: 96 weight: 6 density: 16.0
value: 49 weight: 2 density: 24.5


### 2. Динамичсекое программирование

In [73]:
def fastMaxVal(items, maxWeight, memo={}):
    if items == [] or maxWeight == 0:
        # end of tree
        result = (0, ())
    elif items[0].w > maxWeight:
        # so large
        try:
            result = memo[(len(items[1:]), maxWeight)]
        except KeyError:
            result = fastMaxVal(items[1:], maxWeight, memo)
            memo[(len(items[1:]), maxWeight)] = result
    else:
        # take or don't
        nextItem = items[0]
        
        try:
            withVal, withToTake = memo[(len(items[1:]), maxWeight - nextItem.w)]
        except KeyError:
            withVal, withToTake = fastMaxVal(items[1:], maxWeight - nextItem.w, memo)
            memo[(len(items[1:]), maxWeight - nextItem.w)] = (withVal, withToTake)
        withVal += nextItem.v
        
        try:
            withoutVal, withoutToTake = memo[(len(items[1:]), maxWeight)]
        except KeyError:
            withoutVal, withoutToTake = fastMaxVal(items[1:], maxWeight, memo)
            memo[(len(items[1:]), maxWeight)] = (withoutVal, withoutToTake)
        
        if withVal > withoutVal:
            result = (withVal, withToTake + (nextItem,))
        else:
            result = (withoutVal, withoutToTake)

    return result

In [77]:
%%timeit
fastMaxVal(I, maxWeight)

1.55 µs ± 11.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [74]:
%%time
V = fastMaxVal(I, maxWeight)

CPU times: total: 0 ns
Wall time: 5.5 ms


In [75]:
print(V[0])
for it in V[1]:
    print(it)

617
value: 62 weight: 3 density: 20.666666666666668
value: 94 weight: 4 density: 23.5
value: 73 weight: 5 density: 14.6
value: 90 weight: 5 density: 18.0
value: 84 weight: 3 density: 28.0
value: 69 weight: 2 density: 34.5
value: 96 weight: 6 density: 16.0
value: 49 weight: 2 density: 24.5


### 3. Жадный алгоритм

In [78]:
def greedy(items, maxWeight, keyFunction):
    itemsCopy = sorted(items, key = keyFunction, reverse = True)
    result = []
    totalValue, totalWeight = 0.0, 0.0
    for i in range(len(itemsCopy)):
        if (totalWeight+itemsCopy[i].w) <= maxWeight:
            result.append(itemsCopy[i])
            totalWeight += itemsCopy[i].w
            totalValue += itemsCopy[i].v
    return (result, totalValue)

def testGreedy(items, constraint, keyFunction):
    taken, val = greedy(items, constraint, keyFunction)
    print('Total value of items taken =', val)
    for item in taken:
        print(' ', item)

In [79]:
%%time
V = testGreedy(I, maxWeight, Item.get_density)

Total value of items taken = 614.0
  value: 69 weight: 2 density: 34.5
  value: 84 weight: 3 density: 28.0
  value: 49 weight: 2 density: 24.5
  value: 94 weight: 4 density: 23.5
  value: 62 weight: 3 density: 20.666666666666668
  value: 90 weight: 5 density: 18.0
  value: 70 weight: 4 density: 17.5
  value: 51 weight: 3 density: 17.0
  value: 45 weight: 4 density: 11.25
CPU times: total: 0 ns
Wall time: 1.5 ms


### 4. Случайный перебор